<a href="https://colab.research.google.com/github/abou-zithar/ChatBot_Using_Deeplearning/blob/main/ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from copyreg import pickle
import random
import json
import pickle
import numpy as np 
import nltk

'''
# reduce all words to the stem 
work working worked =>work
'''
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Activation, Dropout
from tensorflow.keras.optimizers import SGD

In [2]:
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


# Load Training Data

In [4]:
Lemmatizer=WordNetLemmatizer()

In [5]:
# reading content as text then path it to json.load
intents=json.loads(open('/content/drive/MyDrive/Chatbot/intents.json').read())

In [6]:
words=[]
classes=[]
documents=[]
ignore_letters=['?','!','.',',']

In [7]:
for intent in intents['intents']:
  for pattern in intent['patterns']:
    word_list =nltk.word_tokenize(pattern)
    words.extend(word_list)
    documents.append((word_list,intent['tag']))
    if intent['tag'] not in classes:
      classes.append(intent['tag'])
print(documents)


[(['Hi', 'there'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hey'], 'greeting'), (['Hola'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['Bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['Nice', 'chatting', 'to', 'you', ',', 'bye'], 'goodbye'), (['Till', 'next', 'time'], 'goodbye'), (['Thanks'], 'thanks'), (['Thank', 'you'], 'thanks'), (['That', "'s", 'helpful'], 'thanks'), (['Awesome', ',', 'thanks'], 'thanks'), (['Thanks', 'for', 'helping', 'me'], 'thanks'), (['How', 'you', 'could', 'help', 'me', '?'], 'options'), (['What', 'help', 'you', 'provide', '?'], 'options'), (['How', 'you', 'can', 'be', 'helpful', '?'], 'options'), (['where', 'is', 'order', 'with', 'id', '431B67', '?'], 'order_tracking'), (['track', 'order', '562B78'], 'order_tracking'), (['Where', 'is', 'my', 'order', 'with', 'id', '561A24', '?'], 'order_tracking'), (['order', 'id', '345A23', 'comprises', '

#Prepare Training Data

In [8]:
words=[Lemmatizer.lemmatize(word.lower()) for word in words if word not in ignore_letters]
words=sorted(set(words))

In [9]:
print(words)

["'s", '23a12', '23a31', '32712', '345a23', '431b67', '561a24', '562b78', '@', 'a', 'abc', 'abx', 'accident', 'ai', 'am', 'an', 'analysis', 'and', 'anyone', 'appointment', 'appoitment', 'are', 'at', 'awesome', 'based', 'be', 'been', 'benefit', 'bhatt', 'bored', 'bye', 'cabin', 'cafeteria', 'call', 'can', 'canteen', 'challenging', 'change', 'chat', 'chatting', 'clarity', 'clear', 'commission', 'company', 'compensation', 'complaint', 'component', 'comprises', 'computer', 'configuration', 'configure', 'conflict', 'cost', 'could', 'customer', 'date', 'day', 'delayed', 'delivery', 'demand', 'department', 'design', 'desktop', 'development', 'do', 'doe', 'doing', 'domain', 'each', 'employee', 'event', 'exit', 'factor', 'feedback', 'find', 'fixed', 'for', 'forgets', 'forgot', 'from', 'gadget', 'gmail.com', 'go', 'good', 'goodbye', 'grossing', 'guide', 'ha', 'handled', 'happy', 'have', 'head', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'highest', 'hola', 'hour', 'how', 'hr/it/projects'

In [10]:
clasees=sorted(set(classes))

In [11]:
print(clasees)

['HR_related_problem', 'Location', 'Weather', 'about', 'appointment status', 'cabin', 'check_leave', 'commission', 'competitors_in_market', 'configuration', 'connect_people', 'cost_lowering', 'customer_satisfaction', 'domain', 'email_id', 'exit', 'factors_impacting_sale', 'forgot_password', 'gadgets', 'goodbye', 'greeting', 'highest_grossing', 'hours', 'invalid', 'key_customers', 'leave', 'maintainence', 'manufacturing_problems', 'missing_id', 'name', 'noans', 'options', 'order_components', 'order_tracking', 'predict_delay', 'predict_performance', 'project_handling_queries', 'search_department', 'search_person_by_id', 'solve_problems', 'supplier_info', 'thanks', 'turnover', 'version_update']


In [12]:
pickle.dump(words,open('words.pk1','wb'))
pickle.dump(classes,open('classes.pk1','wb'))

In [13]:
trainning=[]
output_empty=[0]*len(classes)

In [14]:
print(documents)

[(['Hi', 'there'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hey'], 'greeting'), (['Hola'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['Bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['Nice', 'chatting', 'to', 'you', ',', 'bye'], 'goodbye'), (['Till', 'next', 'time'], 'goodbye'), (['Thanks'], 'thanks'), (['Thank', 'you'], 'thanks'), (['That', "'s", 'helpful'], 'thanks'), (['Awesome', ',', 'thanks'], 'thanks'), (['Thanks', 'for', 'helping', 'me'], 'thanks'), (['How', 'you', 'could', 'help', 'me', '?'], 'options'), (['What', 'help', 'you', 'provide', '?'], 'options'), (['How', 'you', 'can', 'be', 'helpful', '?'], 'options'), (['where', 'is', 'order', 'with', 'id', '431B67', '?'], 'order_tracking'), (['track', 'order', '562B78'], 'order_tracking'), (['Where', 'is', 'my', 'order', 'with', 'id', '561A24', '?'], 'order_tracking'), (['order', 'id', '345A23', 'comprises', '

In [15]:
for doc in documents:
  bag=[]
  word_patterns=doc[0]
  word_patterns=[Lemmatizer.lemmatize(word.lower()) for word in word_patterns]
  for word in words:
    bag.append(1) if word in word_patterns else bag.append(0)

  output_row = list(output_empty)
  output_row[classes.index(doc[1])] = 1
  trainning.append([bag, output_row])



In [16]:
random.shuffle(trainning)
trainning=np.array(trainning)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


In [17]:
train_x=list(trainning[:,0])
train_y=list(trainning[:,1])

In [18]:
print(train_x)


[[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

# Building Model

In [19]:
model=Sequential()
model.add(Dense(128,input_shape=(len(train_x[0],),),activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]),activation='softmax'))

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               31232     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 44)                2860      
                                                                 
Total params: 42,348
Trainable params: 42,348
Non-trainable params: 0
_________________________________________________________________


In [20]:
sgd=SGD(lr=0.01,decay=1e-6,momentum=0.9,nesterov=True)
model.compile(loss='categorical_crossentropy',optimizer=sgd, metrics=["accuracy"])

/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [21]:
hist=model.fit(np.array(train_x),np.array(train_y),epochs=200,batch_size=5,verbose=1)
model.save('ChatBot_model.hs',hist)
print('done')

Epoch 1/200
26/26 [==============================] - 1s 2ms/step - loss: 3.7913 - accuracy: 0.0078 
Epoch 2/200
26/26 [==============================] - 0s 2ms/step - loss: 3.7378 - accuracy: 0.0781
Epoch 3/200
26/26 [==============================] - 0s 2ms/step - loss: 3.6634 - accuracy: 0.0781
Epoch 4/200
26/26 [==============================] - 0s 2ms/step - loss: 3.5353 - accuracy: 0.1016
Epoch 5/200
26/26 [==============================] - 0s 3ms/step - loss: 3.5240 - accuracy: 0.1094
Epoch 6/200
26/26 [==============================] - 0s 2ms/step - loss: 3.4099 - accuracy: 0.1328
Epoch 7/200
26/26 [==============================] - 0s 2ms/step - loss: 3.2535 - accuracy: 0.1484
Epoch 8/200
26/26 [==============================] - 0s 2ms/step - loss: 3.2043 - accuracy: 0.1562
Epoch 9/200
26/26 [==============================] - 0s 2ms/step - loss: 3.0634 - accuracy: 0.1953
Epoch 10/200
26/26 [==============================] - 0s 2ms/step - loss: 2.8996 - accuracy: 0.2656
Epoch 11

# building ChatBot

In [22]:
res=model.predict(np.array(train_x))[0]
results=[[i,r] for i,r in enumerate(res) if r> 0.25]
results.sort(key=lambda x: x[1] ,reverse=True)
results_list=[]
for r in results:
  results_list.append({'intent':classes[r[0]],'probability':str(r[1])})

4/4 [==============================] - 0s 3ms/step


In [23]:
def clean_up_sentance(sentance):
  sentance_words=nltk.word_tokenize(sentance)
  wnl = WordNetLemmatizer()
  sentance_words=[wnl.lemmatize(word) for word in sentance_words]
  return sentance_words


In [24]:
def bag_of_words(sentance):
  sentance_words=clean_up_sentance(sentance)
  bag=[0]*len(words)
  for w in sentance_words:
    for i,word in enumerate(words):
      if word == w:
        bag[i]=1
  return np.array(bag)

      

In [25]:
def predict_class(sentance):
  
  bow = np.array(bag_of_words(sentance))
  res=model.predict(bow.reshape(-1,243))[0]
  ERROR_THRESHHOLD=0.25
  results=[[i,r] for i,r in enumerate(res) if r> ERROR_THRESHHOLD]
  results.sort(key=lambda x: x[1] ,reverse=True)

  results_list=[]
  for r in results:
    results_list.append({'intent':classes[r[0]],'probability':str(r[1])})

  return results_list


In [26]:
def get_response(intents_list, intents_json):
  tag = intents_list[0]['intent']
  list_of_intents= intents_json['intents']
  for i in list_of_intents:
    if i['tag'] ==tag:
      result =random.choice(i['responses'])
      break
  return result

In [27]:
print('___ the BOT is RUNNING......!___')
while True:
  massage=input("")
  ints=predict_class(massage)
  res=get_response(ints,intents)
  print(res)


___ the BOT is RUNNING......!___
hi
1/1 [==============================] - 0s 47ms/step
Hi there, how can I help?
where is my order
1/1 [==============================] - 0s 14ms/step
Please enter with order ID
quit
1/1 [==============================] - 0s 15ms/step
...
stop
1/1 [==============================] - 0s 15ms/step
...


KeyboardInterrupt: ignored